In [ ]:
import mkl

vml_threads=mkl.domain_get_max_threads('vml')
import idi.simulation as sim
import numpy as np
from numpy import pi
from six import print_ as print

!hostname
import matplotlib.pyplot as plt
import numexpr as ne

%matplotlib inline
mkl.domain_set_num_threads(vml_threads,'vml')
import scipy.spatial

### Settings

In [ ]:
Natoms = int(1e7)
Ndet = 1024
detz = 30  # in cm
pixelsize = 50  # in um
Nimg = 10
E = 6400  # in ev
rotangles = np.array([0, 0, 0]) / 180 * pi
cuda = True
r = 20  # nm
simtype = 'multisphere'
rndphase = True
outfile = 'idi20-loose-multi0.npz'
fwhmfocal = 300e-3
spacing = 10e-3

# not used
a = 3.6  # in A # not used for cuso4

### Simulation

In [ ]:
print("preparing")
_a = a * 1e-4  # in um
_r = r * 1e-3  # in um

_detz = detz * 1e4  # in um
k = 2 * pi / (1.24 / E)  # in 1/um
N = Natoms
if simtype == 'gridsc':
    simobject = sim.simobj.gridsc(N, _a, E, rotangles)
elif simtype == 'gridfcc':
    simobject = sim.simobj.gridfcc(N, _a, E, rotangles)
elif simtype == 'gridcuso4':
    simobject = sim.simobj.gridcuso4(N, E, rotangles)
elif simtype == 'multisphere':
    simobject = sim.simobj.multisphere(E=E, Natoms=N, rsphere=_r, fwhmfocal=fwhmfocal, spacing=spacing)
else:
    raise NotImplementedError("unknown object to simulate")
simobject.rndPhase = rndphase
simobject.rndPos = True


if cuda:
    print('using gpu')
    gen = sim.cuda.simulate_gen(simobject, Ndet, pixelsize, _detz, k)
else:
    print('using cpu')
    gen = sim.cpu.simulate_gen(simobject, Ndet, pixelsize, _detz, k)

In [ ]:
print("simulating")


def save(filename):
    np.savez_compressed(
        filename,
        result=np.array(result),
        settings=(
            {
                'Natoms': Natoms,
                'Ndet': Ndet,
                'detz': detz,
                'Nimg': Nimg,
                'a': a,
                'r': r,
                'pixelsize': pixelsize,
                'E': E,
                'rndphase': rndphase,
                'rotangles': rotangles,
                'spacing': spacing,
                'fwhmfocal': fwhmfocal,
            },
            [simtype],
        ),
    )
    print(f'\n saved as {filename}')


import time

lastsave = time.time()
savefile = 0
result = []

for i in range(Nimg):
    print(i, end=" ")
    t = next(gen)
    t = np.abs(t * t.conjugate())
    result.append(t)
    if time.time() - lastsave > 30 * 60:
        of = f'{savefile}-{outfile}'
        save(of)
        savefile = (savefile + 1) % 2
        lastsave = time.time()
save(filename)

In [ ]:
print("done")

## Plots

In [ ]:
plt.matshow(np.log10(np.mean(result, axis=0)), vmax=0)
plt.show()

from idi.util import radial_profile

rad = radial_profile(np.mean(result, axis=0))
rad = rad - np.min(rad)
plt.semilogy(rad[2:-10])
plt.show()

#### Show Simulation object

In [ ]:
pos = simobject.get()
print(simobject._debug)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

fig = plt.figure()
ax = plt.axes(projection='3d')
posf = pos[np.abs(pos[:, 2]) < 0.005]
ax.scatter(posf[:, 0] * 1e4, posf[:, 1] * 1e4, posf[:, 2] * 1e4, s=0.01)
ax.set_zlim(-1000, 1000)
plt.show()

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(pos[:, 0] * 1e4, pos[:, 1] * 1e4, pos[:, 2] * 1e4, s=0.01)
plt.show()